In [3]:
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path: /Users/lunli/Documents/FixedIncomeLib
Fixed Income Library is loaded.


In [4]:
# utilities
def display_res(input_dict : Dict[SimpleMetrics, float]):
    display({k.to_string() : v for k, v in input_dict.items()})

### Test European Option Analytics

In [5]:
### log normal vol
forward = 0.04
strike = 0.042
ln_sigma = 0.20
tte = 0.5
call_or_put = 'put'
res_ln_pricing = qfEuropeanOptionLogNormal(forward, strike, tte, ln_sigma, call_or_put, True)
display_res(res_ln_pricing)

{'pv': 0.0034471895385265877,
 'delta': -0.608068301568095,
 'gamma': 67.92009043331392,
 'vega': 0.010867214469330227,
 'theta': -0.0021734428938660457,
 'strike_risk': 0.6611886095535807}

In [6]:
### b&r test
key = SimpleMetrics.PV
base_pv = res_ln_pricing[key]

epsilon = 1e-6
bump_vec = [0.] * 4
for i in range(len(bump_vec)):
    bump_vec[i] += epsilon    
    bumped_pv = qfEuropeanOptionLogNormal(
        forward + bump_vec[0], 
        strike + bump_vec[1], 
        tte - bump_vec[2], 
        ln_sigma + bump_vec[3], 
        call_or_put)
    display((bumped_pv[key] - base_pv) / epsilon)
    bump_vec[i] -= epsilon

-0.6080343412546985

0.6612194113492231

-0.0021734438519771526

0.010867217570265097

In [7]:
### implied lognormal vol
forward = 0.04
strike = 0.042
pv = res_ln_pricing[SimpleMetrics.PV]
tte = 0.5
res_ln_iv = qfEuropeanOptionImpliedLogNormalVol(pv, forward, strike, tte, call_or_put, True)
display_res(res_ln_iv)

{'implied_log_normal_vol': 0.200000418968941,
 'd_ln_vol_d_forward': 55.954340778786836,
 'd_ln_vol_d_tte': -0.20000041896894094,
 'd_ln_vol_d_strike': -60.84247653659662}

In [8]:
### b&r test
key = SimpleMetrics.IMPLIED_LOG_NORMAL_VOL
base_imp_ln_vol = res_ln_iv[key]

epsilon = 1e-6
bump_vec = [0.] * 3
for i in range(len(bump_vec)):
    bump_vec[i] += epsilon    
    bumped_ln_vol = qfEuropeanOptionImpliedLogNormalVol(
        pv, 
        forward + bump_vec[0], 
        strike + bump_vec[1], 
        tte + bump_vec[2], 
        call_or_put)
    display((bumped_ln_vol[key] - base_imp_ln_vol) / epsilon)
    bump_vec[i] -= epsilon

55.943251391576965

-60.84618949966925

-0.20000011929921513

In [14]:
### normal vol from lognormal vol
forward = 0.04
strike = 0.042
shift = 0.04
ln_vol = res_ln_iv[SimpleMetrics.IMPLIED_LOG_NORMAL_VOL]
tte = 0.5
res_n_vol = qfEuropeanOptionNormalVolFromLogNormalVol(forward, strike, tte, ln_vol, shift, True)
display_res(res_n_vol)

{'implied_normal_vol': 0.016185721892412776,
 'd_n_vol_d_ln_vol': 0.08079364793832611,
 'd_n_vol_d_forward': 0.10074437075552134,
 'd_n_vol_d_strike': 0.09909965833792977,
 'd_n_vol_d_tte': -2.695845471844399e-05}

In [12]:
### b&r test
key = SimpleMetrics.IMPLIED_NORMAL_VOL
base_n_vol = res_n_vol[key]

epsilon = 1e-6
bump_vec = [0.] * 4
for i in range(len(bump_vec)):
    bump_vec[i] += epsilon    
    bumped_n_vol = qfEuropeanOptionNormalVolFromLogNormalVol(
        forward + bump_vec[0], 
        strike + bump_vec[1], 
        tte + bump_vec[2], 
        ln_vol + bump_vec[3],
        shift)
    display((bumped_n_vol[key] - base_n_vol) / epsilon)
    bump_vec[i] -= epsilon

0.10074282869726447

0.09910076599964346

-2.696084674957966e-05

0.08079363500446135

In [16]:
### normal vol
forward = 0.04
strike = 0.042
shift = 0.04
n_sigma = res_n_vol[SimpleMetrics.IMPLIED_NORMAL_VOL]
tte = 0.5
res_nv_pricing = qfEuropeanOptionNormal(forward, strike, tte, n_sigma, call_or_put, True)
display_res(res_nv_pricing)

{'pv': 0.005635445460400294,
 'delta': -0.5693612868156246,
 'gamma': 34.329064804444215,
 'vega': 0.27782034787567483,
 'theta': -0.0044967228867690435,
 'strike_risk': 0.5693612868156246}

In [17]:
### b&r test
key = SimpleMetrics.PV
base_pv = res_nv_pricing[key]

epsilon = 1e-6
bump_vec = [0.] * 4
for i in range(len(bump_vec)):
    bump_vec[i] += epsilon    
    bumped_pv = qfEuropeanOptionNormal(
        forward + bump_vec[0], 
        strike + bump_vec[1], 
        tte - bump_vec[2], 
        n_sigma + bump_vec[3], 
        call_or_put)
    display((bumped_pv[key] - base_pv) / epsilon)
    bump_vec[i] -= epsilon

-0.5693441221957599

0.569378451262259

-0.00449672506695753

0.27782060993755686

In [18]:
### implied normal vol
forward = 0.04
strike = 0.042
pv = res_nv_pricing[SimpleMetrics.PV]
tte = 0.5
res_nv_imp = qfEuropeanOptionImpliedNormalVol(pv, forward, strike, tte, call_or_put, True)
display_res(res_nv_imp)

{'implied_normal_vol': 0.01618572196682995,
 'd_n_vol_d_forward': 2.0493865578010393,
 'd_n_vol_d_tte': -0.016185721966829946,
 'd_n_vol_d_strike': -2.0493865578010393}

In [19]:
### b&r test
key = SimpleMetrics.IMPLIED_NORMAL_VOL
base_imp_n_vol = res_nv_imp[key]

epsilon = 1e-6
bump_vec = [0.] * 3
for i in range(len(bump_vec)):
    bump_vec[i] += epsilon    
    bumped_n_vol = qfEuropeanOptionImpliedNormalVol(
        pv, 
        forward + bump_vec[0], 
        strike + bump_vec[1], 
        tte + bump_vec[2], 
        call_or_put)
    display((bumped_n_vol[key] - base_imp_n_vol) / epsilon)
    bump_vec[i] -= epsilon

2.0492891074086472

-2.04948317678727

-0.01618569769185063

In [20]:
### lognormal vol from normal vol
forward = 0.04
strike = 0.042
shift = 0.04
n_vol = res_nv_imp[SimpleMetrics.IMPLIED_NORMAL_VOL]
tte = 0.5
res_ln_iv_round_trip = qfEuropeanOptionLogNormalVolFromNormalVol( \
    forward, strike, tte, n_vol, shift, True)
display_res(res_ln_iv_round_trip)

{'implied_log_normal_vol': 0.200000424627485,
 'd_ln_vol_d_n_vol': 25.564994731799903,
 'd_ln_vol_d_forward': 19.194946165029986,
 'd_ln_vol_d_strike': -21.668457881185873,
 'd_ln_vol_d_tte': 0.21378747218490063}

In [21]:
### b&r test
key = SimpleMetrics.IMPLIED_LOG_NORMAL_VOL
base_ln_vol = res_ln_iv_round_trip[key]

epsilon = 1e-6
bump_vec = [0.] * 4
for i in range(len(bump_vec)):
    bump_vec[i] += epsilon    
    bumped_ln_vol = qfEuropeanOptionLogNormalVolFromNormalVol(
        forward + bump_vec[0], 
        strike + bump_vec[1], 
        tte + bump_vec[2], 
        n_vol + bump_vec[3],
        shift)
    display((bumped_ln_vol[key] - base_ln_vol) / epsilon)
    bump_vec[i] -= epsilon

-1.2469335680109772

-1.2265576763148545

0.00033364652707312814

12.377211372482932